In [ ]:
import tensorflow as tf
import os
e = "No gpu"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6148)])
  except RuntimeError as e:
    print(e)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
import miscnn

# Creating a Data I/O interface for kidney tumor CT scans in NIfTI format
from miscnn.data_loading.interfaces import NIFTI_interface
interface = NIFTI_interface(pattern="case_00[0-9]*", channels=1, classes=3)
data_path = "/media/rami/New Volume/kits19/data_resized/"
data_io = miscnn.Data_IO(interface, data_path)
sample_list = data_io.get_indiceslist()
sample_list.sort()
print("All samples: " + str(sample_list))

In [ ]:
# Data Augmentation
from miscnn.processing.data_augmentation import Data_Augmentation

# Configurations
data_aug = Data_Augmentation(cycles=2, scaling=True, rotations=True, elastic_deform=True, mirror=True,
                             brightness=True, contrast=True, gamma=True, gaussian_noise=True)

#Subfunctions
from miscnn.processing.subfunctions.normalization import Normalization
from miscnn.processing.subfunctions.clipping import Clipping
from miscnn.processing.subfunctions.resampling import Resampling

# Pixel value normalization Subfunction through Z-Score 
sf_normalize = Normalization(mode='z-score')
# Clipping Subfunction between -79 and 304
sf_clipping = Clipping(min=-79, max=304)
# Resampling Subfunction to voxel spacing 3.22 x 1.62 x 1.62
sf_resample = Resampling((3.22, 1.62, 1.62))

# Creating the list for Subfunctions
subfunctions = [sf_resample, sf_clipping, sf_normalize]

# Preprocessor
from miscnn.processing.preprocessor import Preprocessor
data = data_io

# Configure the Preprocessor class
pp = Preprocessor(data, data_aug=data_aug, batch_size=1, subfunctions=subfunctions, prepare_subfunctions=True, 
                  prepare_batches=False, analysis="patchwise-crop", patch_shape=(80, 80, 80),
                  use_multiprocessing=True)

# Patch overlap for predictions
pp.patchwise_overlap = (40, 40, 40)

# Network Model - Standard 3D Unet
from miscnn.neural_network.model import Neural_Network
#Specify loss functions to import
from miscnn.neural_network.metrics import dice_soft, dice_crossentropy, focal_tversky, background, kidney, tumor, log_cosh_dice_loss, dice_soft_loss
from miscnn.neural_network.architecture.unet.standard import Architecture

unet_standard = Architecture()
# Create the Neural Network model
model = Neural_Network(preprocessor=pp, architecture=unet_standard ,loss= dice_soft_loss, metrics=[dice_soft, dice_crossentropy, background, kidney, tumor],
                       batch_queue_size=1, workers=3, learninig_rate=0.0001)

In [ ]:
# Excluding suspious samples from data set
del sample_list[133]
del sample_list[125]
del sample_list[68]
del sample_list[37]
del sample_list[23]
del sample_list[15]

#Create train sample
train_samples = sample_list[0:160]
# Create the validation sample ID list
validation_samples = sample_list[160:204]
# Create the test sample ID list
test_samples = sample_list[204:294]
# Output validation samples
print("Validation samples: " + str(validation_samples))

**Prediction**

In [ ]:
model.reset_weights()

In [ ]:
model.load('3DUnet_dice_soft.hdf5')

In [ ]:
model.predict(validation_samples)

In [ ]:
from evaluation.kits19_evaluation import kits19_evaluate
l = [int(x[-3:]) for x in validation_samples] 
pred_path = "/media/rami/New Volume/kits19/MIScnn/Pred_Results/dice_loss"
log_path = "/media/rami/New Volume/kits19/MIScnn/"

kits19_evaluate(l, pred_path, log_path)

**Visualize**

In [ ]:
import shutil
import math
import os
from tqdm import tqdm
import numpy as np 
import pandas as pd
import os
import scipy.ndimage
import nibabel as nib
import matplotlib.pyplot as plt
from numpy.random import rand
from skimage import measure, morphology
from scipy.ndimage import zoom
import cv2
import tensorflow as tf

%matplotlib inline

In [ ]:
def hu_to_grayscale(volume, hu_min = -512, hu_max = 512):
    # Clip at max and min values if specified
    if hu_min is not None or hu_max is not None:
        volume = np.clip(volume, hu_min, hu_max)

    # Scale to values between 0 and 1
    mxval = np.max(volume)
    mnval = np.min(volume)
    im_volume = (volume - mnval)/max(mxval - mnval, 1e-3)

    # Return values scaled to 0-255 range, but *not cast to uint8*
    # Repeat three times to make compatible with color overlay
    im_volume = 255*im_volume
    return np.stack((im_volume, im_volume, im_volume), axis=-1)

def class_to_color(segmentation, k_color = [255, 0, 0], t_color = [0, 0, 255]):
    # initialize output to zeros
    shp = segmentation.shape
    seg_color = np.zeros((shp[0], shp[1], shp[2], 3), dtype=np.float32)

    # set output to appropriate color at each location
    seg_color[np.equal(segmentation,1)] = k_color
    seg_color[np.equal(segmentation,2)] = t_color
    return seg_color

def overlay(volume_ims, segmentation_ims, segmentation, alpha):
    # Get binary array for places where an ROI lives
    segbin = np.greater(segmentation, 0)
    repeated_segbin = np.stack((segbin, segbin, segbin), axis=-1)
    # Weighted sum where there's a value to overlay
    overlayed = np.where(
        repeated_segbin,
        np.round(alpha*segmentation_ims+(1-alpha)*volume_ims).astype(np.uint8),
        np.round(volume_ims).astype(np.uint8)
    )
    return overlayed

#Visualize a case after resizing
def Visualize(case, rows, columns, width = 299, height = 299):
    cid = f'{case:05}'
    seg_best = f"/media/rami/New Volume/kits19/MIScnn/Pred_Results/weighted_dice"
    seg_worst = f"/media/rami/New Volume/kits19/MIScnn/Pred_Results/dice_loss"
    orig = f"/media/rami/New Volume/kits19/data_resized/case_{cid}/"
    img = f"/media/rami/New Volume/kits19/data_resized/case_{cid}/"
    img = os.path.join(img, 'imaging.nii.gz') 
    seg_best = os.path.join(seg_best, f'case_{cid}.nii.gz')
    seg_worst = os.path.join(seg_worst, f'case_{cid}.nii.gz')
    
    orig = os.path.join(orig, 'segmentation.nii.gz') 
    img = nib.load(img)
    
    seg_best = nib.load(seg_best)
    seg_worst = nib.load(seg_worst)
    orig = nib.load(orig)
    
    img = img.get_fdata()
    seg_best = seg_best.get_fdata()
    seg_worst = seg_worst.get_fdata()
    orig = orig.get_fdata()
    seg_best = seg_best.astype(np.float32)
    seg_worst = seg_worst.astype(np.float32)
    orig = orig.astype(np.float32)
    img_siz = hu_to_grayscale(img, -width, height)
    seg_best = class_to_color(seg_best, [255, 0, 0], [0, 0, 255])
    seg_worst = class_to_color(seg_worst, [255, 0, 0], [0, 0, 255])
    orig_siz = class_to_color(orig, [255, 0, 0], [0, 0, 255])
    
    plt.subplots(rows, columns,figsize=(15, 15))
    x = 85
    plt.subplot(rows, columns, 2)
    image_pred = overlay(img_siz, seg_best, img, 0.3)
    plt.imshow(image_pred[x], cmap='gray')
    plt.subplot(rows, columns, 1)
    image_orig = overlay(img_siz, orig_siz, img, 0.3)
    plt.imshow(image_orig[x], cmap='gray')
    plt.show()
    
    
rows = 1
columns = 2
case = 189
Visualize(case, rows, columns, 256, 256)